In [279]:
from crawlbase import CrawlingAPI
from bs4 import BeautifulSoup
import pickle
import pandas as pd
import time

def scrape(sub_red):
    token = 'dLN3qq4_CCZeNJTAPTwGdA'
    reddit_url = 'https://www.reddit.com/r/' + sub_red + '/new/'
    api = CrawlingAPI({'token': token})
    response = api.get(reddit_url)

    if response['status_code'] == 200:
        return response['body']
    else:
        print('Error')


In [280]:
def save_post(html_data):    
    with open('model.p', 'rb') as file:
        model = pickle.load(file)
    with open('vectorizer.p', 'rb') as file:
        vectorizer = pickle.load(file)
    soup = BeautifulSoup(html_data, 'html.parser')
    instances = soup.find_all('div', property="schema:articleBody")
    docs = []
    for instance in instances:
        string = str(instance.find_next())
        docs.append(string.strip('</p>\n '))

    verdicts = pd.read_csv('verdicts.csv')
    for doc in docs:
        vectorized = vectorizer.transform([doc])
        predict_probas = model.predict_proba(vectorized)[:, 1]
        threshold = 0.5
        bool = (predict_probas >= threshold)
        row = pd.Series({'post':doc,'verdict':bool})
        if not (verdicts['post'] == doc).any():
            print('saved')
            verdicts = pd.concat([verdicts, pd.DataFrame({'post':doc,'verdict':bool})])
            verdicts.to_csv('verdicts.csv', index=False)
        



In [281]:
sub_reds = ['depression', 'sad', 'horses', 'random']
for sub_red in sub_reds:
    save_post(scrape(sub_red))
    time.sleep(10)

saved
saved
saved


In [ ]:
# verdicts = pd.DataFrame({'post':[], 'verdict':[]})
# verdicts.loc[len(verdicts)] = ['dummy', 'data']
# verdicts.to_csv('verdicts.csv')